In [1]:
# code from: https://qiita.com/ReoNagai/items/a8fdee89b1686ec31d10

#!/usr/bin/env python
# -*- coding: utf-8 -*

import numpy as np
import cv2
from cv2 import aruco
import pandas as pd
import matplotlib.colors as mcolors
import random

random.seed(0)

In [2]:
# debug情報を出すか
IS_DEBUG = False

cap = cv2.VideoCapture("src/mov/test06.mp4")
fps = int(cap.get(cv2.CAP_PROP_FPS))    # 動画のfps
current_frame = 1   # 現在の動画フレーム
# マーカーサイズ
marker_length = 0.0255  # [m]
# マーカーの辞書選択
dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

# キャリブレーションデータの選択
camera_matrix = np.load("calibration/iphonese/mtx.npy")
distortion_coeff = np.load("calibration/iphonese/dist.npy")

# DFの不要な行（後で消す）につけるkey名称
DELETE_KEY = -1

# 軌跡の長さ
orbit_length = 100

# マーカーの履歴（初期状態でマルチカラムのDFをconcatしてもエラーにならないように最低限フォーマットを整える）
markers_hist =  pd.DataFrame([np.nan], columns=[[DELETE_KEY],[0]])

# matplotlibのカラーテーブルを持ってくる（148色分）
colors = list(map(lambda color: tuple(map(lambda c: int(c * 255),mcolors.to_rgb(color))), mcolors.CSS4_COLORS.values()))
random.shuffle(colors)


In [3]:
def apllyFilters(img):
    k = 3
    sharp_kernel = np.array([
        [-k / 9, -k / 9, -k / 9],
        [-k / 9, 1 + 8 * k / 9, k / 9],
        [-k / 9, -k / 9, -k / 9]
    ], np.float32)
    img_sp = cv2.filter2D(img, -1, sharp_kernel).astype("uint8")
    return img_sp


In [4]:
while True:
    ret, img = cap.read()
    if not ret or cv2.waitKey(int(1000 / fps)) == 27:
        break
    # img = apllyFilters(img)

    # マーカー認識
    corners, ids, rejectedImgPoints = aruco.detectMarkers(img, dictionary)
    # マーカー可視化
    aruco.drawDetectedMarkers(img, corners, ids, (0, 255, 255))

    markers_current = pd.DataFrame()
    markers_tmp = []
    if len(corners) > 0:
        # マーカーごとに処理
        for i, corner in enumerate(corners):
            id = ids[i][0]
            this_corner = np.squeeze(corner)

            # 姿勢認識
            # rvec -> rotation vector, tvec -> translation vector
            rvec, tvec, _ = aruco.estimatePoseSingleMarkers(
                corner, marker_length, camera_matrix, distortion_coeff)

            # < rodoriguesからeuluerへの変換 >

            # 不要なaxisを除去
            tvec = np.squeeze(tvec)
            rvec = np.squeeze(rvec)
            # 回転ベクトルからrodoriguesへ変換
            rvec_matrix = cv2.Rodrigues(rvec)
            rvec_matrix = rvec_matrix[0]  # rodoriguesから抜き出し
            # 並進ベクトルの転置
            transpose_tvec = tvec[np.newaxis, :].T
            # 合成
            proj_matrix = np.hstack((rvec_matrix, transpose_tvec))
            # オイラー角への変換
            euler_angle = cv2.decomposeProjectionMatrix(proj_matrix)[
                6]  # [deg]

            # 中心座標(ピクセル単位)
            cpx = int((this_corner[0][0] + this_corner[2][0]) / 2)
            cpy = int((this_corner[0][1] + this_corner[2][1]) / 2)
            # マーカー座標をmarkers(DataFrame)に書き込み
            this_marker = pd.DataFrame([{"x": tvec[0], "y": tvec[1], "z": tvec[2],
                            "roll": euler_angle[0][0], "pitch": euler_angle[1][0], "yaw": euler_angle[2][0],
                            "cpx": cpx, "cpy": cpy
                            }])
            markers_tmp.append(this_marker)
            
            if IS_DEBUG:
                print("marker id: " + str(id))
                for key, val in this_marker.iteritems():
                    print(key + " : " + str(val[0]))
            
            # 可視化
            draw_pole_length = marker_length/2  # 現実での長さ[m]
            cv2.drawFrameAxes(img, camera_matrix, distortion_coeff,
                                rvec, tvec, draw_pole_length)
            
            if i in markers_hist.columns:
                comp = [None, None] # x座標,y座標
                for idx, cp in enumerate(['cpx', 'cpy']):
                    comp[idx] =  markers_hist[i, cp][-orbit_length:]
                    if not pd.isna(comp[idx]).all(): # 全てNaNだと補完できないのでスキップさせる
                        # markers_histからx or yの軌跡を抜き出した配列について、nanの前方の値で補完したあと、後方の値で補完
                        comp[idx] =comp[idx].interpolate("ffill").interpolate("bfill").astype('int')
                IS_DEBUG and print(comp)
                if not pd.isna(comp).any(): # compのいずれもNoneでない場合（＝補完がうまく行った場合）
                    orbit = np.array(list(zip(comp[0], comp[1])))   # x成分とy成分を結合
                    cv2.polylines(img, [orbit], False, colors[i], 1)    # 軌跡を描画
        
        if len(markers_tmp) < 2:    # マーカーが1つのみ検出された場合
            # concatで複数のマーカーDFを結合してマルチカラムを作成しているため、マーカーDFが1つだけの場合
            # 1つのマーカーを複製して無理やりconcatして、複製した方を最後に削除している
            markers_tmp = [markers_tmp[0], markers_tmp[0]]
            markers_current = pd.concat(markers_tmp, axis=1, keys=[ids[0][0], DELETE_KEY ])  # 後で消す方のkey=DELETE_KEY
        else:
            markers_current = pd.concat(markers_tmp, axis=1, keys=np.squeeze(ids))

        markers_hist = pd.concat([markers_hist, markers_current], ignore_index=True)
    else: # マーカーが1つも検出できなかった場合
        markers_hist.loc[current_frame] = np.nan

    current_frame += 1
    cv2.imshow('drawDetectedMarkers', img)

    ###debug
    # if current_frame > 50:
    #     break

# 全フレーム終了後の処理

cv2.destroyAllWindows()

markers_hist.sort_index(level=0, axis=1, inplace=True)
if DELETE_KEY in markers_hist.columns:  # 不要な行を削除
    markers_hist = markers_hist.drop(DELETE_KEY , axis=1)

markers_hist.to_csv("output/test/markers_test.csv")

In [5]:
list(range(1, 4))

[1, 2, 3]